# Baseline

In [126]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, StratifiedGroupKFold, KFold

## Загрузка данных

In [127]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

In [128]:
city_counts = train_df['city'].value_counts().to_dict()
#city_counts
train_df["city_other"] = train_df["city"].apply(lambda x: x  if city_counts.get(x, 0) > 15  else "other")
train_df.drop("city", axis=1, inplace=True)
train_df.city_other.value_counts()

city_to_index_mapping = train_df.dropna(subset=['city_other', 'index_city_code']).drop_duplicates('index_city_code').set_index('index_city_code')['city_other'].to_dict()


for idx, row in train_df.iterrows():
    if pd.isna(row['city_other']):
        index_city_code = row['index_city_code']

        if index_city_code in city_to_index_mapping:
            train_df.at[idx, 'city_other'] = city_to_index_mapping[index_city_code]

        elif not pd.isna(index_city_code):
            train_df.at[idx, 'city_other'] = index_city_code

train_df["city"] = train_df["city_other"]
train_df.drop("city_other", axis=1, inplace=True)

In [129]:

train_df['month_number'] = train_df.groupby('id').cumcount() + 1


train_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city_type,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,ogrn_year,ft_registration_date,max_founderpres,min_founderpres,ogrn_exist_months,okved,segment,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,cnt_a_oper_1m,sum_b_oper_1m,cnt_b_oper_1m,sum_c_oper_1m,cnt_c_oper_1m,sum_deb_d_oper_1m,cnt_deb_d_oper_1m,sum_cred_d_oper_1m,cnt_cred_d_oper_1m,sum_deb_e_oper_1m,cnt_deb_e_oper_1m,cnt_days_deb_e_oper_1m,sum_cred_e_oper_1m,cnt_cred_e_oper_1m,cnt_days_cred_e_oper_1m,sum_deb_f_oper_1m,cnt_deb_f_oper_1m,cnt_days_deb_f_oper_1m,sum_cred_f_oper_1m,cnt_cred_f_oper_1m,cnt_days_cred_f_oper_1m,sum_deb_g_oper_1m,cnt_deb_g_oper_1m,cnt_days_deb_g_oper_1m,sum_cred_g_oper_1m,cnt_cred_g_oper_1m,cnt_days_cred_g_oper_1m,sum_deb_h_oper_1m,cnt_deb_h_oper_1m,cnt_days_deb_h_oper_1m,sum_cred_h_oper_1m,cnt_cred_h_oper_1m,cnt_days_cred_h_oper_1m,sum_a_oper_3m,cnt_a_oper_3m,sum_b_oper_3m,cnt_b_oper_3m,sum_c_oper_3m,cnt_c_oper_3m,sum_deb_d_oper_3m,cnt_deb_d_oper_3m,sum_cred_d_oper_3m,cnt_cred_d_oper_3m,sum_deb_e_oper_3m,cnt_deb_e_oper_3m,cnt_days_deb_e_oper_3m,sum_cred_e_oper_3m,cnt_cred_e_oper_3m,cnt_days_cred_e_oper_3m,sum_deb_f_oper_3m,cnt_deb_f_oper_3m,cnt_days_deb_f_oper_3m,sum_cred_f_oper_3m,cnt_cred_f_oper_3m,cnt_days_cred_f_oper_3m,sum_deb_g_oper_3m,cnt_deb_g_oper_3m,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster,city,month_number
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.452182,2.736461,2.932560,2.437979,okved_30,segment_1,0.942275,0.536013,0.511490,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.396523,0.893064,0.463405,0.505823,0.873795,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,0.099899,0.226247,0.391439,1.054211,0.654057,0.387856,1.171020,0.648942,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.143207,0.870124,-0.084701,0.307935,0.356556,0.426034,1.201541,0.435983,0.864246,1.270517,-0.152107,0.286074,0.891856,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.186820,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other},city_23,1
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.469917,2.756014,2.952725,2.455375,okved_30,segment_1,0.645704,0.536378,0.486425,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.088209,0.969891,-0.063678,0.221993,0.131636,0.893042,0.398889,0.095082,0.873685,0.526503,-0.101797,0.771350,0.244786,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.021465,0.224276,0.326923,0.419979,0.652830,0.258824,0.410410,0.633498,0.564854,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,-0.084701,0.307935,0.302226,0.425993,1.135607,0.411205,0.864251,1.237550,-0.142187,0.286081,0.902845,-0.025646,0.24851,0.038562,-0.040649,0.458055,0.728507,0.125043,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other},city_23,2
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_type_0,index_city_code_39,-0.488553,-0.135063,ogrn_month_0,ogrn_year_1,2.485936,2.773674,2.970940,2.472771,okved_30,segment_1,0.403604,0.613167,0.480547,-0.188835,0.417022,-0.030225,0.435995,-0.103359,0.550798,-0.128513,0.967030,-0.063678,0.221993,0.485924,0.893049,0.560179,0.148894,0.873656,0.623277,-0.125219,0.771335,0.212528,-0.017983,0.154163,0.777702,-0.041418,0.513943,0.848818,-0.029795,0.223291,0.294665,1.251369,0.653444,0.387856,0.552757,0.629637,0.661628,-0.207082,0.208461,-0.029964,0.517298,-0.119654,0.960017,-0.127755,0.870983,

In [130]:
pivot_columns = [col for col in train_df.columns if col not in ['id', 'date', 'month_number']]

df_pivoted = train_df.pivot_table(index='id', columns='month_number', values=pivot_columns, aggfunc='first')

# Теперь, когда мы сформировали сводную таблицу, преобразуем MultiIndex столбцы в единый уровень
df_pivoted.columns = [f'{col[0]}_{col[1]}month' for col in df_pivoted.columns]

# Переименовываем индекс обратно в столбец 'id', если он стал индексом
df_pivoted.reset_index(inplace=True)

df_pivoted.head()



,id,balance_amt_avg_1month,balance_amt_avg_2month,balance_amt_avg_3month,balance_amt_day_avg_1month,balance_amt_day_avg_2month,balance_amt_day_avg_3month,balance_amt_max_1month,balance_amt_max_2month,balance_amt_max_3month,balance_amt_min_1month,balance_amt_min_2month,balance_amt_min_3month,channel_code_1month,channel_code_2month,channel_code_3month,city_1month,city_2month,city_3month,city_type_1month,city_type_2month,city_type_3month,cnt_a_oper_1m_1month,cnt_a_oper_1m_2month,cnt_a_oper_1m_3month,cnt_a_oper_3m_1month,cnt_a_oper_3m_2month,cnt_a_oper_3m_3month,cnt_b_oper_1m_1month,cnt_b_oper_1m_2month,cnt_b_oper_1m_3month,cnt_b_oper_3m_1month,cnt_b_oper_3m_2month,cnt_b_oper_3m_3month,cnt_c_oper_1m_1month,cnt_c_oper_1m_2month,cnt_c_oper_1m_3month,cnt_c_oper_3m_1month,cnt_c_oper_3m_2month,cnt_c_oper_3m_3month,cnt_cred_d_oper_1m_1month,cnt_cred_d_oper_1m_2month,cnt_cred_d_oper_1m_3month,cnt_cred_d_oper_3m_1month,cnt_cred_d_oper_3m_2month,cnt_cred_d_oper_3m_3month,cnt_cred_e_oper_1m_1month,cnt_cred_e_oper_1m_2month,cnt_cred_e_oper_1m_3month,cnt_cred_e_oper_3m_1month,cnt_cred_e_oper_3m_2month,cnt_cred_e_oper_3m_3month,cnt_cred_f_oper_1m_1month,cnt_cred_f_oper_1m_2month,cnt_cred_f_oper_1m_3month,cnt_cred_f_oper_3m_1month,cnt_cred_f_oper_3m_2month,cnt_cred_f_oper_3m_3month,cnt_cred_g_oper_1m_1month,cnt_cred_g_oper_1m_2month,cnt_cred_g_oper_1m_3month,cnt_cred_g_oper_3m_1month,cnt_cred_g_oper_3m_2month,cnt_cred_g_oper_3m_3month,cnt_cred_h_oper_1m_1month,cnt_cred_h_oper_1m_2month,cnt_cred_h_oper_1m_3month,cnt_cred_h_oper_3m_1month,cnt_cred_h_oper_3m_2month,cnt_cred_h_oper_3m_3month,cnt_days_cred_e_oper_1m_1month,cnt_days_cred_e_oper_1m_2month,cnt_days_cred_e_oper_1m_3month,cnt_days_cred_e_oper_3m_1month,cnt_days_cred_e_oper_3m_2month,cnt_days_cred_e_oper_3m_3month,cnt_days_cred_f_oper_1m_1month,cnt_days_cred_f_oper_1m_2month,cnt_days_cred_f_oper_1m_3month,cnt_days_cred_f_oper_3m_1month,cnt_days_cred_f_oper_3m_2month,cnt_days_cred_f_oper_3m_3month,cnt_days_cred_g_oper_1m_1month,cnt_days_cred_g_oper_1m_2month,cnt_days_cred_g_oper_1m_3month,cnt_days_cred_g_oper_3m_1month,cnt_days_cred_g_oper_3m_2month,cnt_days_cred_g_oper_3m_3month,cnt_days_cred_h_oper_1m_1month,cnt_days_cred_h_oper_1m_2month,cnt_days_cred_h_oper_1m_3month,cnt_days_cred_h_oper_3m_1month,cnt_days_cred_h_oper_3m_2month,cnt_days_cred_h_oper_3m_3month,cnt_days_deb_e_oper_1m_1month,cnt_days_deb_e_oper_1m_2month,cnt_days_deb_e_oper_1m_3month,cnt_days_deb_e_oper_3m_1month,cnt_days_deb_e_oper_3m_2month,cnt_days_deb_e_oper_3m_3month,cnt_days_deb_f_oper_1m_1month,cnt_days_deb_f_oper_1m_2month,cnt_days_deb_f_oper_1m_3month,cnt_days_deb_f_oper_3m_1month,cnt_days_deb_f_oper_3m_2month,cnt_days_deb_f_oper_3m_3month,cnt_days_deb_g_oper_1m_1month,cnt_days_deb_g_oper_1m_2month,cnt_days_deb_g_oper_1m_3month,cnt_days_deb_g_oper_3m_1month,cnt_days_deb_g_oper_3m_2month,cnt_days_deb_g_oper_3m_3month,cnt_days_deb_h_oper_1m_1month,cnt_days_deb_h_oper_1m_2month,cnt_days_deb_h_oper_1m_3month,cnt_days_deb_h_oper_3m_1month,cnt_days_deb_h_oper_3m_2month,cnt_days_deb_h_oper_3m_3month,cnt_deb_d_oper_1m_1month,cnt_deb_d_oper_1m_2month,cnt_deb_d_oper_1m_3month,cnt_deb_d_oper_3m_1month,cnt_deb_d_oper_3m_2month,cnt_deb_d_oper_3m_3month,cnt_deb_e_oper_1m_1month,cnt_deb_e_oper_1m_2month,cnt_deb_e_oper_1m_3month,cnt_deb_e_oper_3m_1month,cnt_deb_e_oper_3m_2month,cnt_deb_e_oper_3m_3month,cnt_deb_f_oper_1m_1month,cnt_deb_f_oper_1m_2month,cnt_deb_f_oper_1m_3month,cnt_deb_f_oper_3m_1month,cnt_deb_f_oper_3m_2month,cnt_deb_f_oper_3m_3month,cnt_deb_g_oper_1m_1month,cnt_deb_g_oper_1m_2month,cnt_deb_g_oper_1m_3month,cnt_deb_g_oper_3m_1month,cnt_deb_g_oper_3m_2month,cnt_deb_g_oper_3m_3month,cnt_deb_h_oper_1m_1month,cnt_deb_h_oper_1m_2month,cnt_deb_h_oper_1m_3month,cnt_deb_h_oper_3m_1month,cnt_deb_h_oper_3m_2month,cnt_deb_h_oper_3m_3month,end_cluster_1month,end_cluster_2month,end_cluster_3month,ft_registration_date_1month,ft_registration_date_2month,ft_registration_date_3month,index_city_code_1month,index_city_code_2month,in

In [131]:

X = df_pivoted.drop([
"cnt_a_oper_1m_1month", "cnt_a_oper_1m_2month", "cnt_a_oper_1m_3month",
"cnt_b_oper_1m_1month", "cnt_b_oper_1m_2month", "cnt_b_oper_1m_3month",
"cnt_c_oper_1m_1month", "cnt_c_oper_1m_2month", "cnt_c_oper_1m_3month",
"cnt_deb_d_oper_1m_1month", "cnt_deb_d_oper_1m_2month", "cnt_deb_d_oper_1m_3month",
"cnt_cred_d_oper_1m_1month", "cnt_cred_d_oper_1m_2month", "cnt_cred_d_oper_1m_3month",
"cnt_deb_e_oper_1m_1month", "cnt_deb_e_oper_1m_2month", "cnt_deb_e_oper_1m_3month",
"cnt_cred_e_oper_1m_1month", "cnt_cred_e_oper_1m_2month", "cnt_cred_e_oper_1m_3month",
"cnt_deb_f_oper_1m_1month", "cnt_deb_f_oper_1m_2month", "cnt_deb_f_oper_1m_3month",
"cnt_cred_f_oper_1m_1month", "cnt_cred_f_oper_1m_2month", "cnt_cred_f_oper_1m_3month",
"cnt_deb_g_oper_1m_1month", "cnt_deb_g_oper_1m_2month", "cnt_deb_g_oper_1m_3month",
"cnt_cred_g_oper_1m_1month", "cnt_cred_g_oper_1m_2month", "cnt_cred_g_oper_1m_3month",
"cnt_deb_h_oper_1m_1month", "cnt_deb_h_oper_1m_2month", "cnt_deb_h_oper_1m_3month",
"cnt_cred_h_oper_1m_1month", "cnt_cred_h_oper_1m_2month", "cnt_cred_h_oper_1m_3month",
"cnt_a_oper_3m_1month", "cnt_a_oper_3m_2month", "cnt_a_oper_3m_3month",
"cnt_b_oper_3m_1month", "cnt_b_oper_3m_2month", "cnt_b_oper_3m_3month",
"cnt_c_oper_3m_1month", "cnt_c_oper_3m_2month", "cnt_c_oper_3m_3month",
"cnt_deb_d_oper_3m_1month", "cnt_deb_d_oper_3m_2month", "cnt_deb_d_oper_3m_3month",
"cnt_cred_d_oper_3m_1month", "cnt_cred_d_oper_3m_2month", "cnt_cred_d_oper_3m_3month",
"cnt_deb_e_oper_3m_1month", "cnt_deb_e_oper_3m_2month", "cnt_deb_e_oper_3m_3month",
"cnt_cred_e_oper_3m_1month", "cnt_cred_e_oper_3m_2month", "cnt_cred_e_oper_3m_3month",
"cnt_deb_f_oper_3m_1month", "cnt_deb_f_oper_3m_2month", "cnt_deb_f_oper_3m_3month",
"cnt_cred_f_oper_3m_1month", "cnt_cred_f_oper_3m_2month", "cnt_cred_f_oper_3m_3month",
"cnt_deb_g_oper_3m_1month", "cnt_deb_g_oper_3m_2month", "cnt_deb_g_oper_3m_3month",
"cnt_cred_g_oper_3m_1month", "cnt_cred_g_oper_3m_2month", "cnt_cred_g_oper_3m_3month",
"cnt_deb_h_oper_3m_1month", "cnt_deb_h_oper_3m_2month", "cnt_deb_h_oper_3m_3month",
"cnt_cred_h_oper_3m_1month", "cnt_cred_h_oper_3m_2month", "cnt_cred_h_oper_3m_3month",

"city_type_1month",
"city_type_2month",
"city_type_3month",
"index_city_code_1month",
"index_city_code_2month",
"index_city_code_3month",

"start_cluster_3month",
"id","end_cluster_3month", "end_cluster_2month", "end_cluster_1month"
], axis=1)
y = df_pivoted["end_cluster_3month"]

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [132]:
params_cat = {
    "iterations":10000,
    "depth":5,

    "eval_metric":'AUC',
    "random_seed":1,
    "bootstrap_type":'Bayesian',
    "bagging_temperature":1,
    "od_type":'Iter',
    "od_wait":200,
    "task_type":'GPU'
}

In [133]:
from sklearn.model_selection import StratifiedKFold, LeavePOut, RepeatedStratifiedKFold
from catboost import  CatBoostClassifier


cat_model = CatBoostClassifier(**params_cat)


# cv = KFold(n_splits=7)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=12)

# cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
auc_scores = []  # Список для хранения показателей AUC для каждого разбиения


In [134]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [135]:
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [136]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
for col in cat_features:
    X[col] = X[col].astype('category')
    #test_df[col] = test_df[col].astype('category')



In [137]:
y = y.astype('category')
# y = y.cat.add_categories('Missing').fillna('Missing')

In [138]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
for col in cat_features:
    X[col] = X[col].cat.add_categories('Missing').fillna('Missing')
cat_features

['channel_code_1month',
 'channel_code_2month',
 'channel_code_3month',
 'city_1month',
 'city_2month',
 'city_3month',
 'ogrn_month_1month',
 'ogrn_month_2month',
 'ogrn_month_3month',
 'ogrn_year_1month',
 'ogrn_year_2month',
 'ogrn_year_3month',
 'okved_1month',
 'okved_2month',
 'okved_3month',
 'segment_1month',
 'segment_2month',
 'segment_3month',
 'start_cluster_1month',
 'start_cluster_2month']

In [139]:
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
cat_features

['channel_code_1month',
 'channel_code_2month',
 'channel_code_3month',
 'city_1month',
 'city_2month',
 'city_3month',
 'ogrn_month_1month',
 'ogrn_month_2month',
 'ogrn_month_3month',
 'ogrn_year_1month',
 'ogrn_year_2month',
 'ogrn_year_3month',
 'okved_1month',
 'okved_2month',
 'okved_3month',
 'segment_1month',
 'segment_2month',
 'segment_3month',
 'start_cluster_1month',
 'start_cluster_2month']

In [140]:
X

,balance_amt_avg_1month,balance_amt_avg_2month,balance_amt_avg_3month,balance_amt_day_avg_1month,balance_amt_day_avg_2month,balance_amt_day_avg_3month,balance_amt_max_1month,balance_amt_max_2month,balance_amt_max_3month,balance_amt_min_1month,balance_amt_min_2month,balance_amt_min_3month,channel_code_1month,channel_code_2month,channel_code_3month,city_1month,city_2month,city_3month,cnt_days_cred_e_oper_1m_1month,cnt_days_cred_e_oper_1m_2month,cnt_days_cred_e_oper_1m_3month,cnt_days_cred_e_oper_3m_1month,cnt_days_cred_e_oper_3m_2month,cnt_days_cred_e_oper_3m_3month,cnt_days_cred_f_oper_1m_1month,cnt_days_cred_f_oper_1m_2month,cnt_days_cred_f_oper_1m_3month,cnt_days_cred_f_oper_3m_1month,cnt_days_cred_f_oper_3m_2month,cnt_days_cred_f_oper_3m_3month,cnt_days_cred_g_oper_1m_1month,cnt_days_cred_g_oper_1m_2month,cnt_days_cred_g_oper_1m_3month,cnt_days_cred_g_oper_3m_1month,cnt_days_cred_g_oper_3m_2month,cnt_days_cred_g_oper_3m_3month,cnt_days_cred_h_oper_1m_1month,cnt_days_cred_h_oper_1m_2month,cnt_days_cred_h_oper_1m_3month,cnt_days_cred_h_oper_3m_1month,cnt_days_cred_h_oper_3m_2month,cnt_days_cred_h_oper_3m_3month,cnt_days_deb_e_oper_1m_1month,cnt_days_deb_e_oper_1m_2month,cnt_days_deb_e_oper_1m_3month,cnt_days_deb_e_oper_3m_1month,cnt_days_deb_e_oper_3m_2month,cnt_days_deb_e_oper_3m_3month,cnt_days_deb_f_oper_1m_1month,cnt_days_deb_f_oper_1m_2month,cnt_days_deb_f_oper_1m_3month,cnt_days_deb_f_oper_3m_1month,cnt_days_deb_f_oper_3m_2month,cnt_days_deb_f_oper_3m_3month,cnt_days_deb_g_oper_1m_1month,cnt_days_deb_g_oper_1m_2month,cnt_days_deb_g_oper_1m_3month,cnt_days_deb_g_oper_3m_1month,cnt_days_deb_g_oper_3m_2month,cnt_days_deb_g_oper_3m_3month,cnt_days_deb_h_oper_1m_1month,cnt_days_deb_h_oper_1m_2month,cnt_days_deb_h_oper_1m_3month,cnt_days_deb_h_oper_3m_1month,cnt_days_deb_h_oper_3m_2month,cnt_days_deb_h_oper_3m_3month,ft_registration_date_1month,ft_registration_date_2month,ft_registration_date_3month,max_founderpres_1month,max_founderpres_2month,max_founderpres_3month,min_founderpres_1month,min_founderpres_2month,min_founderpres_3month,ogrn_days_end_month_1month,ogrn_days_end_month_2month,ogrn_days_end_month_3month,ogrn_days_end_quarter_1month,ogrn_days_end_quarter_2month,ogrn_days_end_quarter_3month,ogrn_exist_months_1month,ogrn_exist_months_2month,ogrn_exist_months_3month,ogrn_month_1month,ogrn_month_2month,ogrn_month_3month,ogrn_year_1month,ogrn_year_2month,ogrn_year_3month,okved_1month,okved_2month,okved_3month,segment_1month,segment_2month,segment_3month,start_cluster_1month,start_cluster_2month,sum_a_oper_1m_1month,sum_a_oper_1m_2month,sum_a_oper_1m_3month,sum_a_oper_3m_1month,sum_a_oper_3m_2month,sum_a_oper_3m_3month,sum_b_oper_1m_1month,sum_b_oper_1m_2month,sum_b_oper_1m_3month,sum_b_oper_3m_1month,sum_b_oper_3m_2month,sum_b_oper_3m_3month,sum_c_oper_1m_1month,sum_c_oper_1m_2month,sum_c_oper_1m_3month,sum_c_oper_3m_1month,sum_c_oper_3m_2month,sum_c_oper_3m_3month,sum_cred_d_oper_1m_1month,sum_cred_d_oper_1m_2month,sum_cred_d_oper_1m_3month,sum_cred_d_oper_3m_1month,sum_cred_d_oper_3m_2month,sum_cred_d_oper_3m_3month,sum_cred_e_oper_1m_1month,sum_cred_e_oper_1m_2month,sum_cred_e_oper_1m_3month,sum_cred_e_oper_3m_1month,sum_cred_e_oper_3m_2month,sum_cred_e_oper_3m_3month,sum_cred_f_oper_1m_1month,sum_cred_f_oper_1m_2month,sum_cred_f_oper_1m_3month,sum_cred_f_oper_3m_1month,sum_cred_f_oper_3m_2month,sum_cred_f_oper_3m_3month,sum_cred_g_oper_1m_1month,sum_cred_g_oper_1m_2month,sum_cred_g_oper_1m_3month,sum_cred_g_oper_3m_1month,sum_cred_g_oper_3m_2month,sum_cred_g_oper_3m_3month,sum_cred_h_oper_1m_1month,sum_cred_h_oper_1m_2month,sum_cred_h_oper_1m_3month,sum_cred_h_oper_3m_1month,sum_cred_h_oper_3m_2month,sum_cred_h_oper_3m_3month,sum_deb_d_oper_1m_1month,sum_deb_d_oper_1m_2month,sum_deb_d_oper_1m_3month,sum_deb_d_oper_3m_1month,sum_deb_d_oper_3m_2month,sum_deb_d_oper_3m_3month,sum_deb_e_oper_1m_1month,sum_deb_e_oper_1m_2month,sum_deb_e_oper_1m_3month,sum_deb_e_oper_3m_1month,sum_deb_e_oper_3m_2month,sum_deb_e_oper_3m_3mon

In [141]:
y_val.info()

<class 'pandas.core.series.Series'>
Index: 40000 entries, 119737 to 6584
Series name: end_cluster_3month
Non-Null Count  Dtype 
--------------  ----- 
40000 non-null  object
dtypes: object(1)
memory usage: 625.0+ KB


In [142]:
i=0
for train_idx, val_idx in cv.split(X, y):
    if(i==4)or(i==7)or(i==12)or(i==14):
        print(f"\n\ni: {i} ")
        x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        cat_model.fit(
            x_train, y_train,
            eval_set=(x_val, y_val),
            cat_features=cat_features,
            use_best_model=True,
            verbose=True
        )

        y_pred_proba = cat_model.predict_proba(x_val)
        cat_model.save_model(f'my_meth6_d6_slow/model_{i}.cbm')
        # i+=1
        auc_score = weighted_roc_auc(y_val, y_pred_proba, cat_model.classes_, weights_dict)
        auc_scores.append(auc_score)
    i+=1

mean_auc = np.mean(auc_scores)
std_auc = np.std(auc_scores)
print(f"Средний взвешенный ROC AUC: {mean_auc:.3f} ± {std_auc:.3f}")




i: 4 
Learning rate set to 0.064187


Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7802814	best: 0.7802814 (0)	total: 46.5ms	remaining: 7m 45s
1:	total: 91.6ms	remaining: 7m 38s
2:	total: 134ms	remaining: 7m 25s
3:	total: 181ms	remaining: 7m 31s
4:	total: 222ms	remaining: 7m 23s
5:	test: 0.8177040	best: 0.8177040 (5)	total: 354ms	remaining: 9m 49s
6:	total: 400ms	remaining: 9m 31s
7:	total: 451ms	remaining: 9m 23s
8:	total: 498ms	remaining: 9m 12s
9:	total: 539ms	remaining: 8m 58s
10:	test: 0.8284872	best: 0.8284872 (10)	total: 580ms	remaining: 8m 46s
11:	total: 624ms	remaining: 8m 39s
12:	total: 668ms	remaining: 8m 33s
13:	total: 712ms	remaining: 8m 27s
14:	total: 754ms	remaining: 8m 21s
15:	test: 0.8364916	best: 0.8387189 (13)	total: 797ms	remaining: 8m 17s
16:	total: 841ms	remaining: 8m 13s
17:	total: 890ms	remaining: 8m 13s
18:	total: 934ms	remaining: 8m 10s
19:	total: 982ms	remaining: 8m 9s
20:	test: 0.8418200	best: 0.8420552 (17)	total: 1.03s	remaining: 8m 8s
21:	total: 1.07s	remaining: 8m 7s
22:	total: 1.12s	remaining: 8m 4s
23:	total: 1.16s	remaini

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7775937	best: 0.7775937 (0)	total: 119ms	remaining: 19m 49s
1:	total: 196ms	remaining: 16m 22s
2:	total: 239ms	remaining: 13m 15s
3:	total: 283ms	remaining: 11m 47s
4:	total: 324ms	remaining: 10m 48s
5:	test: 0.8062608	best: 0.8062608 (5)	total: 371ms	remaining: 10m 18s
6:	total: 418ms	remaining: 9m 56s
7:	total: 461ms	remaining: 9m 36s
8:	total: 503ms	remaining: 9m 18s
9:	total: 543ms	remaining: 9m 2s
10:	test: 0.8281730	best: 0.8281730 (10)	total: 585ms	remaining: 8m 51s
11:	total: 630ms	remaining: 8m 44s
12:	total: 678ms	remaining: 8m 40s
13:	total: 724ms	remaining: 8m 36s
14:	total: 771ms	remaining: 8m 33s
15:	test: 0.8334808	best: 0.8339511 (12)	total: 827ms	remaining: 8m 36s
16:	total: 872ms	remaining: 8m 31s
17:	total: 917ms	remaining: 8m 28s
18:	total: 964ms	remaining: 8m 26s
19:	total: 1.01s	remaining: 8m 23s
20:	test: 0.8389886	best: 0.8389886 (20)	total: 1.06s	remaining: 8m 21s
21:	total: 1.1s	remaining: 8m 21s
22:	total: 1.15s	remaining: 8m 18s
23:	total: 1.19s	r

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7714977	best: 0.7714977 (0)	total: 47.7ms	remaining: 7m 56s
1:	total: 92.2ms	remaining: 7m 40s
2:	total: 132ms	remaining: 7m 21s
3:	total: 176ms	remaining: 7m 20s
4:	total: 221ms	remaining: 7m 20s
5:	test: 0.8084383	best: 0.8084383 (5)	total: 271ms	remaining: 7m 31s
6:	total: 314ms	remaining: 7m 28s
7:	total: 358ms	remaining: 7m 27s
8:	total: 406ms	remaining: 7m 30s
9:	total: 450ms	remaining: 7m 29s
10:	test: 0.8324618	best: 0.8324618 (10)	total: 494ms	remaining: 7m 28s
11:	total: 543ms	remaining: 7m 31s
12:	total: 599ms	remaining: 7m 40s
13:	total: 649ms	remaining: 7m 42s
14:	total: 694ms	remaining: 7m 42s
15:	test: 0.8371682	best: 0.8371682 (15)	total: 741ms	remaining: 7m 42s
16:	total: 787ms	remaining: 7m 42s
17:	total: 831ms	remaining: 7m 41s
18:	total: 879ms	remaining: 7m 41s
19:	total: 922ms	remaining: 7m 40s
20:	test: 0.8386998	best: 0.8389565 (19)	total: 969ms	remaining: 7m 40s
21:	total: 1.02s	remaining: 7m 42s
22:	total: 1.06s	remaining: 7m 42s
23:	total: 1.11s	rem

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7276863	best: 0.7276863 (0)	total: 174ms	remaining: 29m 4s
1:	total: 349ms	remaining: 29m 7s
2:	total: 425ms	remaining: 23m 37s
3:	total: 466ms	remaining: 19m 24s
4:	total: 508ms	remaining: 16m 56s
5:	test: 0.7790859	best: 0.7790859 (5)	total: 547ms	remaining: 15m 10s
6:	total: 588ms	remaining: 14m
7:	total: 628ms	remaining: 13m 4s
8:	total: 668ms	remaining: 12m 21s
9:	total: 709ms	remaining: 11m 48s
10:	test: 0.8029007	best: 0.8029007 (10)	total: 752ms	remaining: 11m 23s
11:	total: 792ms	remaining: 10m 58s
12:	total: 838ms	remaining: 10m 43s
13:	total: 882ms	remaining: 10m 29s
14:	total: 925ms	remaining: 10m 15s
15:	test: 0.8104223	best: 0.8105716 (14)	total: 968ms	remaining: 10m 4s
16:	total: 1.01s	remaining: 9m 52s
17:	total: 1.05s	remaining: 9m 42s
18:	total: 1.09s	remaining: 9m 31s
19:	total: 1.13s	remaining: 9m 23s
20:	test: 0.8180493	best: 0.8180493 (20)	total: 1.18s	remaining: 9m 18s
21:	total: 1.22s	remaining: 9m 12s
22:	total: 1.26s	remaining: 9m 5s
23:	total: 1.3s